In [0]:
  --INSERT /*+APPEND PARALLEL(8)*/ INTO stg_sma_case_phase_properties_theCase
    SELECT cidla, json_value(JSONDATA, '$.theCase.casePhase' returning varchar2(120)) as casephase, timestamp  FROM gap_catalog.ads_src_20250901.DLK_ADS_SMA_MONITOR_EVENTS_FULL  s
    left join ads_owner.cases c on c.case_source_id=s.cidla and c.case_valid_to=date'3000-01-01' and c.case_source_system_id='SMA' and c.case_source_sys_origin='SMA_MONITOR_EVENTS' and s.type='CASE_TRANSITIONS_EVALUATED'
    left join ads_owner.case_status cs on c.cstat_key=cs.cstat_key and cs.cstat_valid_to=date'3000-01-01'
                where s.type in ('CASE_CREATED',
        'CASE_COMPLETED',
        'CASE_WAITING',
        'CASE_RUNNING',
        'CASE_UPDATED',
        'CASE_PHASE_CHANGED',
        'CASE_RECONCILE',
        'CASE_TRANSITIONS_EVALUATED'
    --,     --JVE:  zmena typu case
              ) 
              and json_value(JSONDATA, '$.theCase.casePhase') is not null and (c.case_key is null or cs.CSTAT_SOURCE_ID=json_value(jsondata, '$.theCase.caseStatus')) 
;
---------------------------------------------------------------------------------------------
SELECT 
    s.cidla, 
    get_json_object(s.JSONDATA, '$.theCase.casePhase') as casephase, 
    s.timestamp  
FROM gap_catalog.ads_src_20250901.DLK_ADS_SMA_MONITOR_EVENTS_FULL s
LEFT JOIN gap_catalog.ads_src_20250901.cases c 
    ON c.case_source_id = s.cidla 
    AND c.case_valid_to = DATE('3000-01-01') 
    AND c.case_source_system_id = 'SMA' 
    AND c.case_source_sys_origin = 'SMA_MONITOR_EVENTS' 
    AND s.type = 'CASE_TRANSITIONS_EVALUATED'
LEFT JOIN gap_catalog.ads_src_20250901.case_status cs 
    ON c.cstat_key = cs.cstat_key 
    AND cs.cstat_valid_to = DATE('3000-01-01')
WHERE SYS_EFFECTIVE_DATE = '2025-09-01'
  and s.type IN (
        'CASE_CREATED',
        'CASE_COMPLETED',
        'CASE_WAITING',
        'CASE_RUNNING',
        'CASE_UPDATED',
        'CASE_PHASE_CHANGED',
        'CASE_RECONCILE',
        'CASE_TRANSITIONS_EVALUATED'
    ) 
    AND get_json_object(s.JSONDATA, '$.theCase.casePhase') IS NOT NULL 
    AND (
        c.case_key IS NULL 
        OR cs.CSTAT_SOURCE_ID = get_json_object(s.jsondata, '$.theCase.caseStatus')
    );
---- Verze 2
SELECT 
    s.cidla, 
    CAST(get_json_object(s.JSONDATA, '$.theCase.casePhase') AS STRING) as casephase, 
    s.timestamp  
FROM (select * from gap_catalog.ads_src_20250901.DLK_ADS_SMA_MONITOR_EVENTS_FULL s
      WHERE SYS_EFFECTIVE_DATE = '2025-09-01') s
LEFT JOIN gap_catalog.ads_src_20250901.cases c 
    ON c.case_source_id = s.cidla 
    AND c.case_valid_to = TO_DATE('3000-01-01', 'yyyy-MM-dd') 
    AND c.case_source_system_id = 'SMA' 
    AND c.case_source_sys_origin = 'SMA_MONITOR_EVENTS' 
    AND s.type = 'CASE_TRANSITIONS_EVALUATED'
LEFT JOIN gap_catalog.ads_src_20250901.case_status cs 
    ON c.cstat_key = c :param_1 s.cstat_key 
    AND cs.cstat_valid_to = TO_DATE('3000-01-01', 'yyyy-MM-dd')
WHERE s.type IN (
        'CASE_CREATED',
        'CASE_COMPLETED',
        'CASE_WAITING',
        'CASE_RUNNING',
        'CASE_UPDATED',
        'CASE_PHASE_CHANGED',
        'CASE_RECONCILE',
        'CASE_TRANSITIONS_EVALUATED'
    ) 
    AND get_json_object(s.JSONDATA, '$.theCase.casePhase') IS NOT NULL 
    AND (
        c.case_key IS NULL 
        OR cs.CSTAT_SOURCE_ID = get_json_object(s.jsondata, '$.theCase.caseStatus')
    );
